In [ ]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              222KB  2021-04-02 05:44:50           1563  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44           1045  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            850  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19            380  
alsgroup/end-als                                            End ALS Kaggle C

In [ ]:
! kaggle datasets download -d sebastiaanjohn/sneakers

100% 8.46G/8.47G [03:17<00:00, 129MB/s]
100% 8.47G/8.47G [03:17<00:00, 46.0MB/s]


In [ ]:
! unzip sneakers.zip -d /content/Kaggle_sneakers_dataset

Data augmentations: random horizontal flip, random rotation, and random perspective

In [ ]:
xform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
horiz = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.RandomHorizontalFlip(1)])
rotate = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.RandomRotation(0,360)])
perspective = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.RandomPerspective(distortion_scale=0.5, p=1.0, interpolation=2, fill=0)])

horiz_flip_dataset = datasets.ImageFolder('/content/Kaggle_sneakers_dataset/sneaker_dataset/images/train', transform=horiz)
rotate_dataset = datasets.ImageFolder('/content/Kaggle_sneakers_dataset/sneaker_dataset/images/train', transform=rotate)
perspective_dataset = datasets.ImageFolder('/content/Kaggle_sneakers_dataset/sneaker_dataset/images/train', transfor=perspective)
dataset_full = horiz_flip_dataset + rotate_dataset + perspective_dataset

In [ ]:
# use dataset to make dataset_full here
n_all = len(dataset_full)
n_train = int(0.8 * n_all)
n_test = n_all - n_train
rng = torch.Generator().manual_seed(1549)
# should probably split a different way since we're working with a bunch of catagories, might have all images in one catagory entirely in dataset_train for example
dataset_train, dataset_test = torch.utils.data.random_split(dataset_full, [n_train, n_test], rng)

# label the photos, only works for 2 categories
#dataset_train_01 = [(sample, 1 if label == 1 else 0) for (sample, label) in dataset_train if label in [1,2]]
#dataset_test_01 = [(sample, 1 if label == 1 else 0) for (sample, label) in dataset_test if label in [1,2]]

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size = 4, shuffle=True)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size = 4, shuffle=True)

In [ ]:
model = models.resnet18(pretrained=True)
categories = 899 
layer1 = torch.nn.Linear(model_layer.fc.in_features, 1000)
layer2 = torch.nn.Linear(1000, 900)
layer3 = torch.nn.Linear(900, categories) 
torch.nn.init.xavier_uniform_(layer1.weight)
torch.nn.init.xavier_uniform_(layer2.weight)
torch.nn.init.xavier_uniform_(layer3.weight)

model_layer.fc = nn.Sequential(
          layer1,
          nn.ReLU(),
          layer2,
          nn.ReLU(),
          layer3
        )


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Parameter containing:
tensor([[ 0.0298,  0.0318,  0.0437,  ..., -0.0366, -0.0127,  0.0109],
        [ 0.0058,  0.0225, -0.0002,  ...,  0.0416, -0.0383, -0.0218],
        [ 0.0186,  0.0545,  0.0247,  ...,  0.0631, -0.0167,  0.0554],
        ...,
        [ 0.0505, -0.0632,  0.0282,  ...,  0.0165, -0.0461, -0.0257],
        [-0.0134, -0.0206, -0.0193,  ...,  0.0599, -0.0440,  0.0579],
        [ 0.0499,  0.0547, -0.0305,  ..., -0.0193,  0.0186, -0.0422]],
       requires_grad=True)

In [ ]:
device = torch.device('cuda:0')
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

def run_test(model):
    nsamples_test = len(dataset_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [ ]:
def run_train(model, opt, sched):
    nsamples_train = len(dataset_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

In [ ]:
def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")